In [1]:
import pandas as pd
import os, sys
import nltk
sys.path.append(".")
import preprocessing.cleaning_functions
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

cwd = os.getcwd()
data_path = os.path.join(cwd, "data")

### Using stock data

In [2]:
cwd = os.getcwd()
data_path = os.path.join(cwd, "data")

news_df = pd.read_csv(os.path.join(data_path, "RedditNews.csv"))
stock_df = pd.read_csv(os.path.join(data_path, "upload_DJIA_table.csv"))
# snp_500 = pd.read_csv(os.path.join(data_path, "S&P 500 Historical Data.csv"))

In [3]:
df = preprocessing.cleaning_functions.clean_and_combine(stock_df, news_df)

The labels correspond to a change in stock value as follows:

0. change < -2%
1. -2% < change < 0%
2. 0% < change < 2%
3. 2% < change

In [4]:
train = df[df["Date"] < datetime.datetime(2015, 1, 1)]
test = df[df["Date"] > datetime.datetime(2014, 12, 31)]

In [5]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row, 3]))

In [6]:
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

(1611, 31674)


In [7]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.7, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1611, 529)


In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

advancedmodel = MultinomialNB(alpha=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["change_cat"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds4 = advancedmodel.predict(advancedtest)
acc4=accuracy_score(test['change_cat'], preds4)

In [9]:
print('NBayes 1 accuracy: ', acc4)

NBayes 1 accuracy:  0.5079365079365079


Deep Learning

In [10]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\mathi\.conda\envs\nltk_tf\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
C:\Users\mathi\.conda\envs\nltk_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 

In [11]:
max_features = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1
maxlen = 200
batch_size = 32
nb_classes = 2

In [12]:
tokenizer = Tokenizer(nb_words=max_features)
tokenizer.fit_on_texts(trainheadlines)
sequences_train = tokenizer.texts_to_sequences(trainheadlines)
sequences_test = tokenizer.texts_to_sequences(testheadlines)

C:\Users\mathi\.conda\envs\nltk_tf\lib\site-packages\keras_preprocessing\text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [13]:
import numpy as np

y_train = np.array(train["change_cat"])
y_test = np.array(test["change_cat"])

In [14]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)


IndexError: index 2 is out of bounds for axis 1 with size 2

## New project with tripadvisor data

In [3]:
tripadvisor_df = pd.read_csv(os.path.join(data_path,"tripadvisor_hotel_reviews.csv"))
tripadvisor_df["Rating"].value_counts()

5    9054
4    6039
3    2184
2    1793
1    1421
Name: Rating, dtype: int64